In [1]:
# import shutil
# shutil.copy("/kaggle/input/vehicle-detection-v1/training_log.csv", "/kaggle/working/training_log.csv")

In [2]:
%%writefile ddp.py

import csv
import os
import torch
import torchvision
import torchmetrics
import numpy as np
import torchvision.transforms.functional as F
from PIL import Image
from typing import Dict, List, Tuple
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm
from torch.optim import AdamW, SGD
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.transform import GeneralizedRCNNTransform
from torchvision.models.detection.backbone_utils import BackboneWithFPN
from torchvision.models import mobilenet_v3_small
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
import math
import torchvision.models as models
from torchvision.models import MobileNet_V3_Small_Weights
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Import DDP related modules
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import functools


data_transform = transforms.Compose([
    transforms.Resize(size=(640, 640)),
    transforms.ToTensor()
])

class_to_idx = {
    "bus": 0, "car": 1, "motorbike": 2, "truck": 3,
}


class CustomDataset(Dataset):
    def __init__(self,
                 root_dir,
                 split="train",
                 class_to_idx=class_to_idx,
                 transform=data_transform):
        self.root_dir = root_dir
        self.split = split
        self.transform = transform
        self.class_to_idx = class_to_idx if class_to_idx else {}

        self.image_dir = os.path.join(root_dir, "images", split)
        self.label_dir = os.path.join(root_dir, "labels", split)

        self.image_files = [f for f in os.listdir(
            self.image_dir) if f.endswith(".jpg") or f.endswith(".png")]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_filename = self.image_files[idx]
        img_path = os.path.join(self.image_dir, img_filename)

        img_name, _ = os.path.splitext(img_filename)
        label_path = os.path.join(self.label_dir, img_name + ".txt")

        img = Image.open(img_path).convert("RGB")
        original_width, original_height = img.size

        boxes = []
        labels = []

        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue

                    class_id = int(parts[0])
                    x_center, y_center, w, h = map(float, parts[1:])

                    xmin = (x_center - w / 2) * original_width
                    ymin = (y_center - h / 2) * original_height
                    xmax = (x_center + w / 2) * original_width
                    ymax = (y_center + h / 2) * original_height

                    if xmin < xmax and ymin < ymax:
                        boxes.append([xmin, ymin, xmax, ymax])
                        labels.append(class_id + 1)

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels}

        if len(boxes) == 0:
            boxes = torch.zeros((1, 4), dtype=torch.float32)
            labels = torch.zeros((1,), dtype=torch.int64)

        if self.transform:
            img = self.transform(img)

        new_width, new_height = 640, 640
        scale_x = new_width / original_width
        scale_y = new_height / original_height

        boxes[:, [0, 2]] *= scale_x
        boxes[:, [1, 3]] *= scale_y
        target["boxes"] = boxes

        return img, target


class CustomRCNNTransform(GeneralizedRCNNTransform):
    def __init__(self):
        super().__init__(min_size=640, max_size=640, image_mean=[
            0.485, 0.456, 0.406], image_std=[0.229, 0.224, 0.225])

    def resize(self, image, target):
        image = F.resize(image, [640, 640])

        if target is not None and "boxes" in target:
            w_old, h_old = image.shape[-1], image.shape[-2]
            w_new, h_new = 640, 640
            scale_w = w_new / w_old
            scale_h = h_new / h_old
            target["boxes"][:, [0, 2]] *= scale_w
            target["boxes"][:, [1, 3]] *= scale_h
        return image, target


class FRCNN(torch.nn.Module):
    def __init__(self,
                 num_classes,
                 pretrained=MobileNet_V3_Small_Weights.DEFAULT):
        super(FRCNN, self).__init__()
        self.num_classes = num_classes
        self.backbone = self.get_backbone(pretrained)

        self.anchor_sizes = (32, 64, 128, 256)
        self.aspect_ratios = ((0.5, 1.0, 2.0),) * len(self.anchor_sizes)

        self.anchor_generator = AnchorGenerator(
            sizes=self.anchor_sizes,
            aspect_ratios=self.aspect_ratios
        )

        self.model = FasterRCNN(
            backbone=self.backbone,
            num_classes=num_classes,
            rpn_anchor_generator=self.anchor_generator
        )

        self.model.transform = CustomRCNNTransform()

    def get_backbone(self, pretrained):
        backbone = mobilenet_v3_small(weights=pretrained).features
        """Chọn lớp 2, 7, 12 trong backbone vì:
        - Cung cấp đặc trưng ở các mức độ phân giải và độ sâu khác nhau
        - Có số kênh đầu ra (24, 48, 576) phù hợp với thiết kế của FPN trong mã của bạn
        - Phân bố đều trong cấu trúc của MobileNetV3-Small để tối ưu hóa việc phát hiện đối tượng ở nhiều tỷ lệ"""
        return_layers = {'2': '0', '7': '1', '12': '2'}
        in_channels = [24, 48, 576]

        backbone.out_channels = 64
        fpn = BackboneWithFPN(
            backbone=backbone,
            return_layers=return_layers,
            in_channels_list=in_channels,
            out_channels=64
        )

        return fpn

    def forward(self, images, targets=None):
        if self.training:
            if targets is None:
                raise ValueError("In training mode, targets should be passed")
            return self.model(images, targets)
        else:
            return self.model(images)



# Define a custom collate function outside other functions
def detection_collate_fn(batch):
    return tuple(zip(*batch))


# Training loop with DDP - defined as top-level function
def train_one_epoch(model, optimizer, train_loader, device, rank):
    model.train()
    total_train_loss = 0.0
    total_loc_loss = 0.0  # Localization Loss
    total_cls_loss = 0.0  # Classification Loss
    
    if rank == 0:
        progress_bar = tqdm(train_loader, desc="Training", leave=False)
    else:
        progress_bar = train_loader
    
    for images, targets in progress_bar:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()
        loss_dict = model(images, targets)
        
        # Trích xuất các thành phần loss
        loc_loss = loss_dict["loss_box_reg"] + loss_dict.get("loss_rpn_box_reg", 0.0)  # Localization Loss
        cls_loss = loss_dict["loss_classifier"]  # Classification Loss
        total_loss = sum(loss for loss in loss_dict.values())
        
        total_train_loss += total_loss.item()
        total_loc_loss += loc_loss.item()
        total_cls_loss += cls_loss.item()
        
        total_loss.backward()
        optimizer.step()
    
    # Tính trung bình qua tất cả batch
    train_loss = total_train_loss / len(train_loader)
    train_loc_loss = total_loc_loss / len(train_loader)
    train_cls_loss = total_cls_loss / len(train_loader)
    
    # Đồng bộ hóa giữa các GPU
    train_loss_tensor = torch.tensor(train_loss, device=device)
    train_loc_loss_tensor = torch.tensor(train_loc_loss, device=device)
    train_cls_loss_tensor = torch.tensor(train_cls_loss, device=device)
    dist.all_reduce(train_loss_tensor, op=dist.ReduceOp.SUM)
    dist.all_reduce(train_loc_loss_tensor, op=dist.ReduceOp.SUM)
    dist.all_reduce(train_cls_loss_tensor, op=dist.ReduceOp.SUM)
    
    world_size = dist.get_world_size()
    train_loss = train_loss_tensor.item() / world_size
    train_loc_loss = train_loc_loss_tensor.item() / world_size
    train_cls_loss = train_cls_loss_tensor.item() / world_size
    
    return train_loss, train_loc_loss, train_cls_loss

# Evaluation function - unchanged
def evaluate(model, val_loader, device, rank):
    model.eval()
    metric = MeanAveragePrecision(class_metrics=True, extended_summary=True).to(device)
    total_val_loss = 0.0
    total_loc_loss = 0.0  # Localization Loss
    total_cls_loss = 0.0  # Classification Loss
    
    if rank == 0:
        progress_bar = tqdm(val_loader, desc="Evaluating", leave=False)
    else:
        progress_bar = val_loader

    with torch.no_grad():
        for images, targets in progress_bar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            model.train()  # Temporarily enable training mode for loss
            loss_dict = model(images, targets)
            
            # Trích xuất các thành phần loss
            loc_loss = loss_dict["loss_box_reg"] + loss_dict.get("loss_rpn_box_reg", 0.0)  # Localization Loss
            cls_loss = loss_dict["loss_classifier"]  # Classification Loss
            total_loss = sum(loss for loss in loss_dict.values())
            
            total_val_loss += total_loss.item()
            total_loc_loss += loc_loss.item()
            total_cls_loss += cls_loss.item()
            
            model.eval()  # Back to eval mode
            outputs = model(images)
            preds = [
                {"boxes": output["boxes"], "scores": output["scores"], "labels": output["labels"]}
                for output in outputs
            ]
            metric.update(preds, targets)

    # Tính toán kết quả từ metric
    results = metric.compute()
    map50 = torch.tensor(results["map_50"].item(), device=device)
    map95 = torch.tensor(results["map"].item(), device=device)
    map95_per_class = results["map_per_class"][results["map_per_class"] != -1].tolist()
    
    # Trích xuất precision và recall từng class
    iou_idx = 0  # IoU=0.5
    num_classes = len(map95_per_class)
    precision_tensor = results["precision"][iou_idx]  # Shape: (R, K, A, M)
    recall_tensor = results["recall"][iou_idx]  # Shape: (K, A, M)
    
    precision_per_class = [precision_tensor[:, cls, 0, :].mean().item() if not torch.isnan(precision_tensor[:, cls, 0, :]).all() else 0.0 for cls in range(num_classes)]
    recall_per_class = [recall_tensor[cls, 0, :].mean().item() if not torch.isnan(recall_tensor[cls, 0, :]).all() else 0.0 for cls in range(num_classes)]
    
    precision_avg = sum(precision_per_class) / len(precision_per_class) if precision_per_class else 0.0
    recall_avg = sum(recall_per_class) / len(recall_per_class) if recall_per_class else 0.0
    
    # Tính trung bình loss
    val_loss = total_val_loss / len(val_loader)
    val_loc_loss = total_loc_loss / len(val_loader)
    val_cls_loss = total_cls_loss / len(val_loader)
    
    # Đồng bộ hóa giữa các GPU
    dist.all_reduce(map50, op=dist.ReduceOp.SUM)
    dist.all_reduce(map95, op=dist.ReduceOp.SUM)
    dist.all_reduce(torch.tensor(val_loss, device=device), op=dist.ReduceOp.SUM)
    dist.all_reduce(torch.tensor(val_loc_loss, device=device), op=dist.ReduceOp.SUM)
    dist.all_reduce(torch.tensor(val_cls_loss, device=device), op=dist.ReduceOp.SUM)
    
    map95_per_class_tensor = torch.tensor(map95_per_class, device=device)
    precision_per_class_tensor = torch.tensor(precision_per_class, device=device)
    recall_per_class_tensor = torch.tensor(recall_per_class, device=device)
    dist.all_reduce(map95_per_class_tensor, op=dist.ReduceOp.SUM)
    dist.all_reduce(precision_per_class_tensor, op=dist.ReduceOp.SUM)
    dist.all_reduce(recall_per_class_tensor, op=dist.ReduceOp.SUM)
    
    world_size = dist.get_world_size()
    map50 = map50.item() / world_size
    map95 = map95.item() / world_size
    val_loss = val_loss / world_size
    val_loc_loss = val_loc_loss / world_size
    val_cls_loss = val_cls_loss / world_size
    map95_per_class = (map95_per_class_tensor / world_size).tolist()
    precision_per_class = (precision_per_class_tensor / world_size).tolist()
    recall_per_class = (recall_per_class_tensor / world_size).tolist()
    precision_avg = precision_avg / world_size
    recall_avg = recall_avg / world_size

    return map50, map95, val_loss, map95_per_class, precision_per_class, recall_per_class, precision_avg, recall_avg, val_loc_loss, val_cls_loss

# Main DDP training function with ReduceLROnPlateau
def train_ddp(rank, world_size, data_dir, num_classes, num_epochs, batch_size, lr, resume_from):
    # Setup DDP
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    
    device = torch.device(f"cuda:{rank}")
    torch.cuda.set_device(device)

    # Initialize datasets and samplers
    train_dataset = CustomDataset(data_dir)
    val_dataset = CustomDataset(data_dir, split="val")

    train_sampler = DistributedSampler(
        train_dataset, 
        num_replicas=world_size, 
        rank=rank,
        shuffle=True
    )
    
    val_sampler = DistributedSampler(
        val_dataset, 
        num_replicas=world_size, 
        rank=rank,
        shuffle=False
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=2,
        collate_fn=detection_collate_fn,
        pin_memory=True
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        sampler=val_sampler,
        num_workers=2,
        collate_fn=detection_collate_fn,
        pin_memory=True
    )

    # Create model
    model = FRCNN(num_classes=num_classes)
    model = model.to(device)
    model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[rank], output_device=rank)

    # Create optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=lr,               # Initial learning rate (passed as argument)
        betas=(0.85, 0.999), # Adjusted beta1 for DDP
        weight_decay=0.01,
        eps=1e-8
    )
    
    # Create ReduceLROnPlateau scheduler (only on rank 0)
    scheduler = None
    if rank == 0:
        scheduler = ReduceLROnPlateau(
            optimizer,
            mode='min',       # Minimize validation loss
            factor=0.1,       # Reduce lr by 10x
            patience=5,       # Wait 5 epochs
            threshold=0.0001, # Minimum improvement
            min_lr=1e-6,      # Minimum learning rate
            verbose=True      # Print updates
        )

    # Load checkpoint if resuming
    best_map95 = 0
    start_epoch = 1
    
    if resume_from and os.path.exists(resume_from):
        map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
        checkpoint = torch.load(resume_from, map_location=map_location, weights_only=True)
        model.module.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        best_map95 = checkpoint.get("best_map95", 0)
        if rank == 0 and "scheduler_state_dict" in checkpoint:
            scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        if rank == 0:
            print(f"Resuming from epoch {start_epoch} with best map95: {best_map95:.4f}")

    
    # Định nghĩa tên class
    class_names = ["bus", "car", "motorbike", "truck"]

    # CSV logging chỉ trên rank 0
    csv_filename = "training_log.csv"
    if rank == 0:
        if not os.path.exists(csv_filename) or not resume_from:
            with open(csv_filename, "w", newline="") as f:
                writer = csv.writer(f)
                headers = ["epoch", "train_loss", "train_loc_loss", "train_cls_loss", "val_loss", "val_loc_loss", "val_cls_loss", 
                           "mAP50", "mAP95", "precision_avg", "recall_avg"]
                for cls in class_names:
                    headers.append(f"mAP95_{cls}")
                    headers.append(f"precision_{cls}")
                    headers.append(f"recall_{cls}")
                headers.append("learning_rate")
                writer.writerow(headers)
    
    # Training loop
    for epoch in range(start_epoch, num_epochs + 1):
        train_sampler.set_epoch(epoch)
        val_sampler.set_epoch(epoch)
        
        train_loss, train_loc_loss, train_cls_loss = train_one_epoch(model, optimizer, train_loader, device, rank)
        map50, map95, val_loss, map95_per_class, precision_per_class, recall_per_class, precision_avg, recall_avg, val_loc_loss, val_cls_loss = evaluate(model, val_loader, device, rank)
        
        if rank == 0 and scheduler is not None:
            scheduler.step(val_loss)
        
        if rank == 0:
            checkpoint = {
                "epoch": epoch,
                "model_state_dict": model.module.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "best_map95": best_map95,
                "scheduler_state_dict": scheduler.state_dict() if scheduler else None
            }
            torch.save(checkpoint, "last_model.pt")
            
            if map95 > best_map95:
                best_map95 = map95
                torch.save(checkpoint, "best_model.pt")
            
            current_lr = optimizer.param_groups[0]['lr']
            with open(csv_filename, "a", newline="") as f:
                writer = csv.writer(f)
                row = [epoch, train_loss, train_loc_loss, train_cls_loss, val_loss, val_loc_loss, val_cls_loss, 
                       map50, map95, precision_avg, recall_avg]
                row.extend(map95_per_class)
                row.extend(precision_per_class)
                row.extend(recall_per_class)
                row.append(current_lr)
                writer.writerow(row)
            
            print(f"[Epoch {epoch}/{num_epochs}] "
                  f"Train Loss: {train_loss:.4f} (Loc: {train_loc_loss:.4f}, Cls: {train_cls_loss:.4f}) | "
                  f"Val Loss: {val_loss:.4f} (Loc: {val_loc_loss:.4f}, Cls: {val_cls_loss:.4f}) | "
                  f"mAP@50: {map50:.4f} | mAP@50:95: {map95:.4f} | "
                  f"Precision (Avg): {precision_avg:.4f} | Recall (Avg): {recall_avg:.4f} | LR: {current_lr:.6f}")
            for cls, m95, prec, rec in zip(class_names, map95_per_class, precision_per_class, recall_per_class):
                print(f"  {cls}: mAP@50:95: {m95:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f}")
    
    dist.destroy_process_group()


# Single GPU training function
def train_single_gpu(data_dir, num_classes, num_epochs, batch_size, lr, resume_from=None):
    model = FRCNN(num_classes = num_classes).to(device)
    optimizer = SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=0.0005)
    
    # best_val_loss = float("inf")  # Đổi từ best_loss sang best_val_loss
    best_map95 = 0
    start_epoch = 1
    
    if resume_from and os.path.exists(resume_from):
        map_location = {'cuda:%d' % 0: 'cuda:%d' % rank}
        checkpoint = torch.load(resume_from, map_location=map_location, weights_only=True)
        model.module.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        # best_map95 = checkpoint.get("best_map95", 0) 
        print(f"Resuming from epoch {start_epoch} with best map95: {best_map95:.4f}")
    
    # Sửa header của CSV để thêm val_loss
    csv_filename = "training_log.csv"
    with open(csv_filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["epoch", "train_loss", "val_loss", "mAP50", "mAP95"])
    
    for epoch in range(start_epoch, num_epochs + 1):
        # Train
        model.train()
        total_train_loss = 0.0
        progress_bar = tqdm(train_loader, desc="Training", leave=False)
        for images, targets in progress_bar:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(images, targets)
            loss = sum(loss for loss in loss_dict.values())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())
        
        train_loss = total_train_loss / len(train_loader)
        
        # Evaluate với validation loss
        model.eval()
        metric = MeanAveragePrecision().to(device)
        total_val_loss = 0.0
        with torch.no_grad():
            for images, targets in tqdm(val_loader, desc="Evaluating", leave=False):
                images = [img.to(device) for img in images]
                targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
                # Tính validation loss
                model.train()  # Tạm thời bật training mode để tính loss
                loss_dict = model(images, targets)
                loss = sum(loss for loss in loss_dict.values())
                total_val_loss += loss.item()
                model.eval()  # Quay lại eval mode
                
                outputs = model(images)
                preds = [
                    {"boxes": output["boxes"], "scores": output["scores"], "labels": output["labels"]}
                    for output in outputs
                ]
                metric.update(preds, targets)
        
        val_loss = total_val_loss / len(val_loader)
        results = metric.compute()
        map50 = results["map_50"].item()
        map95 = results["map"].item()
        
        # Save checkpoints
        torch.save({
                    "epoch": epoch,
                    "model_state_dict": model.module.state_dict(),
                    "optimizer_state_dict": optimizer.state_dict(),
                    "best_map95": best_map95# Sửa key
                }, "last_model.pt")
        
        if map95 > best_map95:
                    best_map95 = map95
                    torch.save({
                        "epoch": epoch,
                        "model_state_dict": model.module.state_dict(),
                        "optimizer_state_dict": optimizer.state_dict(),
                        "best_map95": best_map95
                    }, "best_model.pt")
        
        # Log cả train_loss và val_loss
        with open(csv_filename, "a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([epoch, train_loss, val_loss, map50, map95])
        
        print(f"[Epoch {epoch}/{num_epochs}] Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | mAP@50: {map50:.4f} | mAP@50:95: {map95:.4f}")


def main():
    # Set params
    world_size = torch.cuda.device_count()  # Number of GPUs available
    data_dir = '/kaggle/input/vehicle-detection-v1/combination/combination'
    num_classes = 5
    num_epochs = 50
    batch_size = 64
    lr = 1e-4
    resume_from = None
    
    if world_size > 1:
        print(f"Training with {world_size} GPUs using DDP")
        # Adjust batch size per GPU
        batch_size_per_gpu = batch_size // world_size
        
        # Use spawn method for starting processes
        mp.spawn(
            train_ddp,
            args=(world_size, data_dir, num_classes, num_epochs, batch_size_per_gpu, lr, resume_from),
            nprocs=world_size,
            join=True
        )
    else:
        print("Only one GPU detected, training without DDP")
        train_single_gpu(data_dir, num_classes, num_epochs, batch_size, lr, resume_from)


if __name__ == "__main__":
    # Ensure proper process initialization for CUDA
    mp.set_start_method('spawn', force=True)
    main()

Writing ddp.py


In [3]:
!python ddp.py

Training with 2 GPUs using DDP
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
Downloading: "https://download.pytorch.org/models/mobilenet_v3_small-047dcff4.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_small-047dcff4.pth
100%|██████████████████████████████████████| 9.83M/9.83M [00:00<00:00, 89.1MB/s]
100%|██████████████████████████████████████| 9.83M/9.83M [00:00<00:00, 85.5MB/s]
[Epoch 1/50] Train Loss: 0.6962 (Loc: 0.3023, Cls: 0.2228) | Val Loss: 0.2919 (Loc: 0.1669, Cls: 0.1017) | mAP@50: 0.6147 | mAP@50:95: 0.3086 | Precision (Avg): 0.2340 | Recall (Avg): 0.3164 | LR: 0.000100
  bus: mAP@50:95: 0.2428 | Precision: 0.4478 | Recall: 0.6961
  car: mAP@50:95: 0.4087 | Precision: 0.5267 | Recall: 0.6095
  motorbike: mAP@50:95: 0.3481 | Precision: 0.4600 | Recall: 0.5515
  truck: mAP@50:95: 0.2346 | Precision: 0.4376 | Recall: 0.6742
[Epoch 2/50] Train Loss: 0.5337 (Loc: 0